In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import catboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [27]:
df = pd.read_csv('./data/DatasetV4+.csv')
df.set_index('Client ID', inplace=True)

In [28]:
df

,Gender,Year,Trip Type,Order Type,Country,Star Rate,Group Type,Meal Option,Destination City,Departure City,...,lat_from,lng_from,lat_to,lng_to,Birthday,Order,Order Date,Trip Start Date,Price,Booking Depth
Client ID,,,,,,,,,,,,,,,,,,,,,
4197429,1,3,0,0,0,2,0,1,0,1,...,59.939043,30.315816,36.719216,31.568803,672537600.0,4803671,1.612731e+09,1.620086e+09,1718.64,85.136111
4197517,1,3,0,0,0,1,0,2,20,0,...,55.752044,37.618500,25.079911,55.147475,342230400.0,4803915,1.612775e+09,1.614816e+09,948.45,23.622917
4197719,1,3,0,0,0,1,0,2,8,0,...,55.752044,37.618500,25.079911,55.147475,-39052800.0,4806034,1.612814e+09,1.633738e+09,2128.15,242.176389
4199375,1,3,0,0,0,1,0,2,0,0,...,55.752044,37.618500,25.079911,55.147475,-300758400.0,4807039,1.612875e+09,1.614384e+09,993.85,17.468056
4208819,1,3,0,0,0,3,2,5,31,0,...,55.752044,37.618500,25.079911,55.147475,484704000.0,4808190,1.612892e+09,1.622938e+09,1121.02,116.265278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141955,0,3,4,0,2,4,1,3,80,0,...,55.752044,37.618500,25.079911,55.147475,160704000.0,4778962,1.611759e+09,1.612570e+09,1164.77,9.376389
4141837,0,3,0,0,0,2,3,1,0,1,...,59.939043,30.315816,36.719216,31.568803,335577600.0,4778919,1.611759e+09,1.619395e+09,468.29,88.384028
4142747,0,3,0,0,0,2,1,1,19,0,...,55.752044,37.618500,25.079911,55.147475,805334400.0,4779312,1.611766e+09,1.625789e+09,1008.25,162.303472


# Цена

In [57]:
X = df.loc[:, df.columns != 'Price']
y = df['Price']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(162140, 23) (162140,)


In [58]:
model = catboost.CatBoostRegressor()
model.fit(X_train, y_train, verbose=500)

Learning rate set to 0.092792
0:	learn: 1620.2081366	total: 13.5ms	remaining: 13.5s
500:	learn: 685.4495442	total: 5.5s	remaining: 5.48s
999:	learn: 614.1768552	total: 10.6s	remaining: 0us


In [41]:
model.predict(X_test)

array([5337.58697916, 2760.01220912,  776.88271645, ..., 1533.07369912,
       1022.81594485, 3178.19533885])

In [61]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = model.predict(X_test)

rmse = mean_absolute_error(y_test.values, preds)
# print(np.sqrt(rmse))
rmse
# model.eval_metrics(X_test, metrics=['MAE', 'RMSE'])

416.7255787433235

In [51]:
y_test.values

array([4367.  , 2845.23,  942.77, ..., 1230.6 ,  801.  , 3001.07])

# Тип питания

In [62]:
X = df.loc[:, df.columns != 'Meal Option']
y = df['Meal Option']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(162140, 23) (162140,)


In [63]:
clf = catboost.CatBoostClassifier()
clf.fit(X_train, y_train, verbose=500)

Learning rate set to 0.101411
0:	learn: 1.6345281	total: 113ms	remaining: 1m 52s
500:	learn: 0.2781199	total: 41.7s	remaining: 41.6s
999:	learn: 0.2316647	total: 1m 21s	remaining: 0us


In [64]:
from sklearn.metrics import accuracy_score

preds = clf.predict(X_test)
acc = accuracy_score(y_test, preds)
acc

0.890650055507586

In [69]:
d = {}
for item in preds:
    for pred in item:
        if pred not in d:
            d[pred] = 1
        else:
            d[pred] += 1

In [71]:
X_test.shape

(32428, 23)

# Звездность

In [72]:
X = df.loc[:, df.columns != 'Star Rate']
y = df['Star Rate']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(162140, 23) (162140,)


In [74]:
clf_star = catboost.CatBoostClassifier()
clf._star.fit(X_train, y_train, verbose=500)

Learning rate set to 0.101411
0:	learn: 1.5252660	total: 80.5ms	remaining: 1m 20s
500:	learn: 0.2067484	total: 34.8s	remaining: 34.7s
999:	learn: 0.1698442	total: 1m 11s	remaining: 0us


In [ ]:
from sklearn.metrics import accuracy_score

preds = clf_star.predict(X_test)
acc = accuracy_score(y_test, preds)
acc

# Продолжительность

In [86]:
X = df.loc[:, df.columns != 'N Nights']
y = df['N Nights']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

(162140, 23) (162140,)


In [89]:
clf_nights = catboost.CatBoostRegressor()
clf_nights.fit(X_train, y_train, verbose=500)

Learning rate set to 0.085345
0:	learn: 3.7583508	total: 14.3ms	remaining: 14.3s
500:	learn: 3.0459909	total: 3.86s	remaining: 3.85s
999:	learn: 2.9094843	total: 8.15s	remaining: 0us


In [88]:
from sklearn.metrics import accuracy_score

preds = clf_nights.predict(X_test)
acc = accuracy_score(y_test, preds)
acc

0.3798815838164549

In [103]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = clf_nights.predict(X_test)
round_preds = np.around(preds)

rmse = mean_absolute_error(y_test.values, round_preds)
# rmse = mean_squared_error(y_test.values, round_preds)
# print(np.sqrt(rmse))
rmse
# model.eval_metrics(X_test, metrics=['MAE', 'RMSE'])

2.353472307882077

# Курорт

In [9]:
# -- // --

## Use class

In [29]:
from modniy_class import Reccommender

In [30]:
rec = Reccommender()

In [ ]:
report = rec.fit(train=df.dropna())

In [83]:
def get_user_pred(_id):
    d = {}
    
    _user = df.loc[[_id]].drop(columns='Price')
    d['reg_price'] = rec.reg_price.predict(_user)
    
    _user = df.loc[[_id]].drop(columns='Meal Option')
    d['clf_meal'] = rec.clf_meal.predict(_user)[0]
    
    _user = df.loc[[_id]].drop(columns='Star Rate')
    d['clf_star'] = rec.clf_star.predict(_user)[0]
    
    _user = df.loc[[_id]].drop(columns='N Nights')
    d['reg_dur'] = np.round(rec.reg_dur.predict(_user))
    
    return d

In [92]:
def get_users_pred(df):
    ids = df.index.values
    pred = pd.DataFrame(index=df.index)
    
    users = df.drop(columns='Price')
    pred['reg_price'] = rec.reg_price.predict(users)
    
    users = df.drop(columns='Meal Option')
    pred['clf_meal'] = rec.clf_meal.predict(users)
    
    users = df.drop(columns='Star Rate')
    pred['clf_star'] = rec.clf_star.predict(users)
    
    users = df.drop(columns='N Nights')
    pred['reg_dur'] = np.round(rec.reg_dur.predict(users))
    
    return pred

In [95]:
preds = get_users_pred(df)

In [96]:
preds

,reg_price,clf_meal,clf_star,reg_dur
Client ID,,,,
4197429,2615.451279,1,1,3.0
4197517,1143.598901,2,1,6.0
4197719,2777.547691,2,1,6.0
4199375,1322.732137,2,1,1.0
4208819,1423.836085,5,3,5.0
...,...,...,...,...
4141955,580.222171,3,4,5.0
4141837,547.634125,1,1,4.0
4142747,972.142858,1,2,1.0
